# Installing System Packages

In [4]:
import sys
print(sys.executable)


/home/swastikgupta/.conda/envs/law/bin/python


In [10]:
import sys
!{sys.executable} -m pip install pandas --no-cache-dir


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 4.7 MB/s  0:00:024.6 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pandas]━━━━━━━━━━━ 2/3 [pandas]


In [5]:
import sys
!{sys.executable} -m pip install langchain-google-genai --no-cache-dir

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11/11 [langchain-google-genai]32m 9/11 [google-genai]


In [18]:
import sys
!{sys.executable} -m pip install langchain-core langchain-text-splitters --no-cache-dir


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [33]:
!pip install langchain --no-deps


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached langchain-1.2.0-py3-none-any.whl.metadata (4.9 kB)
Using cached langchain-1.2.0-py3-none-any.whl (102 kB)


# Importing Google_Api_key_using_(.env)

In [10]:
from dotenv import load_dotenv
import os

load_dotenv()

google_api_key = os.getenv("GOOGLE_API_KEY")

In [11]:
if google_api_key:
    print("✅ GOOGLE_API_KEY loaded")
else:
    print("❌ GOOGLE_API_KEY NOT found")

✅ GOOGLE_API_KEY loaded


# Langchain LLM Test

In [12]:


from langchain_google_genai import ChatGoogleGenerativeAI

# Use the latest stable model version
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash", # or "gemini-2.5-pro" for complex tasks
    temperature=0.7
)

response = llm.invoke("Explain transformers in simple terms")
print(response.content)

/home/swastikgupta/.conda/envs/law/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Imagine you're reading a long, complex sentence and trying to understand it fully:

"The animal didn't cross the street because **it** was too tired."

To understand what "**it**" refers to, your brain automatically looks at the whole sentence, particularly the word "animal." You understand that "it" refers to the animal, not the street.

**Transformers are like a super-smart computer program that learns to do this automatically and much, much better.**

Here's the simple breakdown:

1.  **The Problem They Solve:** Old computer programs for language often processed words one by one, sequentially. It was hard for them to remember early parts of a long sentence or understand how distant words related to each other (like "animal" and "it"). This meant they struggled with context.

2.  **The Big Idea: "Attention" (The Magic Sauce):**
    *   When a Transformer processes a word (like "it"), it doesn't just look at that word in isolation.
    *   It **looks at *every other word* in the sente

# High Level Architecture



# RAG Pipeline

## Step 0: We have our Knowlege in the form of Markdown files (.md)

## Step 1: Loading the .md files.

### LangChain works with a standard object called Document.

Each Document =
📄 text + 🏷️ metadata

In [7]:
from langchain_community.document_loaders import TextLoader
from pathlib import Path

docs = []

for md_file in Path("Markdown Files").glob("*.md"):
    loader = TextLoader(str(md_file), encoding="utf-8")
    docs.extend(loader.load())


In [8]:
print(docs[0].page_content[:1000])

![Image](


In [11]:
import pandas as pd

# 1. Convert the list of Document objects into a list of dictionaries
data = []
for doc in docs:
    data.append({
        "Source": doc.metadata.get("source"),
        "Metadata": doc.metadata,
        "Content Preview": doc.page_content[:200].replace("\n", " ") + "..." # Truncate for readability
    })

# 2. Create the DataFrame
df = pd.DataFrame(data)

# 3. Display the table
# Using head() to avoid a giant table if you have hundreds of files
df.head()

,Source,Metadata,Content Preview
0,Markdown Files/Indian_Constitution.md,{'source': 'Markdown Files/Indian_Constitution...,"![Image](data:image/png;base64,iVBORw0KGgoAAAA..."
1,Markdown Files/Indian_Criminal_Law.md,{'source': 'Markdown Files/Indian_Criminal_Law...,"![Image](data:image/png;base64,iVBORw0KGgoAAAA..."
2,Markdown Files/Indian_Criminal_Law_2.md,{'source': 'Markdown Files/Indian_Criminal_Law...,"![Image](data:image/png;base64,iVBORw0KGgoAAAA..."
3,Markdown Files/Indian_Criminal_Law_3.md,{'source': 'Markdown Files/Indian_Criminal_Law...,"![Image](data:image/png;base64,iVBORw0KGgoAAAA..."
4,Markdown Files/Contract_Laws.md,{'source': 'Markdown Files/Contract_Laws.md'},SECTIONS PREAMBLE 1. Short title. Extent. C...


## Step 2: Cleaning Docling Markdown

In [ ]:
import re

def clean_markdown_for_legal_chunking(text):
    # 1. Remove Images & Docling OCR noise (Do this FIRST)
    text = re.sub(r'!\[([^\]]*)\]\([^\)]+\)', '', text) # Standard MD images
    text = re.sub(r'!Image\s+[^\n]+', '', text)        # OCR "!Image" tags
    
    # 2. Remove Links but keep text
    text = re.sub(r'\[([^\]]+)\]\([^\)]+\)', r'\1', text)
    
    # 3. Remove HTML tags
    text = re.sub(r'<[^>]+>', '', text)
    
    # 4. Remove Mojibake/Scrambled Noise
    text = re.sub(r'£ÉÉ®iÉ|BÉEÉ|ºÉÆÉÊ´ÉvÉÉxÉ', '', text)
    
    # 5. Normalize Whitespace
    text = re.sub(r'[ \t]+', ' ', text)
    text = re.sub(r'\n{3,}', '\n\n', text)

    # 6. MODIFIED Part E: 
    # Remove ONLY bold/italic markers (*, _) and dividers (---), 
    # but EXPLICITLY KEEP the # symbols for headers.
    text = re.sub(r'[*_]{1,}', '', text)  # Remove bold/italic stars and underscores
    text = re.sub(r'[-]{3,}', '', text)   # Remove horizontal lines (---)
    
    return text.strip()

    

In [ ]:
for doc in docs:
    doc.page_content = clean_markdown_for_legal_chunking(doc.page_content)

In [ ]:
data1 = []
for doc in docs:
    data1.append({
        "Source": doc.metadata.get("source"),
        "Metadata": doc.metadata,
        "Content Preview": doc.page_content[:200].replace("\n", " ") + "..." # Truncate for readability
    })

# 2. Create the DataFrame
df1 = pd.DataFrame(data1)

# 3. Display the table
# Using head() to avoid a giant table if you have hundreds of files
df1.head()

## Step 3: Chunking the docs for Embeddings

## Actual Chunking Logic

In [12]:
import re
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from langchain_core.documents import Document

# 1. Define the Cleaning Function (Fixed for the "i" issue)
def clean_legal_md(text):
    text = re.sub(r'!\[([^\]]*)\]\([^\)]+\)', '', text) # Remove MD Images
    text = re.sub(r'!Image\s+[^\n]+', '', text)        # Remove OCR "Image" tags
    text = re.sub(r'\[([^\]]+)\]\([^\)]+\)', r'\1', text) # Remove links, keep text
    # Clean only non-alphabet/non-Hindi symbols (Keep 'i')
    text = re.sub(r'[£É®ºÆÉÊvÉBÉEÉ]', '', text) 
    text = re.sub(r'[ \t]+', ' ', text)
    text = re.sub(r'\n{3,}', '\n\n', text)
    return text.strip()

# 2. Setup the Splitters
# Stage 1: Structural (Header) Splitter
headers = [("#", "Law_Title"), ("##", "Section_Article")]
header_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers)

# Stage 2: Character (Safety) Splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=150,
    separators=["\n\n", "\n", ". ", " "]
)

# 3. The Execution Loop
all_final_documents = []

for doc in docs:
    # A. Clean the text
    cleaned_content = clean_legal_md(doc.page_content)
    
    # B. Extract Law Name from filename (e.g., 'ipc.md' -> 'IPC')
    file_path = doc.metadata.get("source", "Unknown_Law")
    law_name = file_path.split("/")[-1].replace(".md", "").upper()
    
    # C. Split by Headers first (This creates the initial Document objects)
    header_docs = header_splitter.split_text(cleaned_content)
    
    # D. Sub-split large sections and inherit metadata
    # The 'split_documents' method keeps the Law_Title/Section_Article metadata!
    final_chunks = text_splitter.split_documents(header_docs)
    
    # E. Add File Source to Metadata
    for chunk in final_chunks:
        chunk.metadata["law_type"] = law_name
        chunk.metadata["file_source"] = file_path
        all_final_documents.append(chunk)

print(f"Total Chunks Created: {len(all_final_documents)}")

Total Chunks Created: 3367


In [13]:
# Checking Chunks info
import numpy as np

chunk_lengths = [len(c.page_content) for c in all_final_documents]
print(f"Average Chunk Size: {np.mean(chunk_lengths):.2f} characters")
print(f"Max Chunk Size: {max(chunk_lengths)} characters")
print(f"Min Chunk Size: {min(chunk_lengths)} characters")

Average Chunk Size: 777.33 characters
Max Chunk Size: 1008 characters
Min Chunk Size: 1 characters


In [14]:
for i, chunk in enumerate(all_final_documents[:30]):
    print(f"--- CHUNK {i+1} ---")
    print(f"METADATA: {chunk.metadata}")
    print(f"CONTENT PREVIEW: {chunk.page_content[:200]}...") # First 200 chars
    print("-" * 30)

--- CHUNK 1 ---
METADATA: {'Section_Article': 'i ´x', 'law_type': 'INDIAN_CONSTITUTION', 'file_source': 'Markdown Files/Indian_Constitution.md'}
CONTENT PREVIEW: [ 1 , 2024 ]...
------------------------------
--- CHUNK 2 ---
METADATA: {'Section_Article': 'TH CONSTITUTION OF INDIA', 'law_type': 'INDIAN_CONSTITUTION', 'file_source': 'Markdown Files/Indian_Constitution.md'}
CONTENT PREVIEW: [ As on 1 st May , 2024]  
2024  
GOVRNMNT OF INDIA MINISTRY OF LAW AND JUSTIC LGISLATIV DPARTMNT, OFFICIAL LANGUAGS WING...
------------------------------
--- CHUNK 3 ---
METADATA: {'Section_Article': 'PRFAC', 'law_type': 'INDIAN_CONSTITUTION', 'file_source': 'Markdown Files/Indian_Constitution.md'}
CONTENT PREVIEW: This is the sixth pocket size edition of the Constitution of India in the diglot form. In this edition, the text of the Constitution of India has been brought up-to-date by incorporating therein all t...
------------------------------
--- CHUNK 4 ---
METADATA: {'Section_Article': 'PRFAC', 

## Step 4: Setting up Embadding 

## Embadding Logic

In [16]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cpu"}
)

print(len(embeddings.embed_query("Indian Penal Code")))


384


In [17]:
type(all_final_documents[0])

NameError: name 'all_final_documents' is not defined

## Step 5: Creaing VectoreStore FIASS

In [14]:
from langchain_community.vectorstores import FAISS



In [15]:
vectorstore = FAISS.from_documents(
    documents=all_final_documents,
    embedding=embeddings
)

NameError: name 'all_final_documents' is not defined

## Step 6: Saving the Vetor Store

In [23]:
vectorstore.save_local("law_faiss_store")


## Step 7: Creating a Retreaver 

In [29]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 6}
)


## Step 8: Creating a Google Gemini(llm) Instense for Law with temperature 0.2

In [30]:
llm_law = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash", # or "gemini-2.5-pro" for complex tasks
    temperature=0.2
)

## Step 9: Creating a Memory for Chats

In [2]:
import langchain
print(f"Version: {langchain.__version__}")
print(f"Path: {langchain.__file__}")

Version: 1.2.0
Path: /home/swastikgupta/.conda/envs/law/lib/python3.11/site-packages/langchain/__init__.py


In [5]:
def detect_law_domain(text: str):
    """
    Detect relevant Indian law domain based on keywords.
    Returns a law_type string matching your vector metadata.
    """

    text = text.lower()

    domain_keywords = {
        "IPC": [
            "murder", "theft", "assault", "cheating", "fraud",
            "criminal", "rape", "kidnapping", "hurt"
        ],
        "CRPC": [
            "arrest", "police", "bail", "fir", "investigation",
            "custody", "warrant"
        ],
        "CPC": [
            "civil suit", "injunction", "property dispute",
            "recovery", "damages"
        ],
        "FAMILY": [
            "divorce", "maintenance", "alimony",
            "domestic violence", "custody", "dowry"
        ],
        "LABOUR": [
            "termination", "salary", "notice period",
            "employment", "wages", "gratuity"
        ]
    }

    for domain, keywords in domain_keywords.items():
        if any(keyword in text for keyword in keywords):
            return domain

    return None


In [6]:
def update_case_summary(llm, old_summary, user_input, assistant_response):
    """
    Update the running legal case summary using the LLM.
    """

    prompt = f"""
You are a legal case summarizer.

Existing case summary:
{old_summary}

New interaction:
User: {user_input}
Assistant: {assistant_response}

Task:
Update the case summary by:
- Keeping only legally relevant facts
- Removing repetition
- Writing in third person
- Keeping it concise
"""

    summary = llm.invoke(prompt).content.strip()

    return summary


In [22]:
def legal_assistant_chat(
    user_query: str,
    vectorstore,
    llm,
    case_summary: str = ""
):
    """
    Chatbot-style legal assistant function.

    Args:
        user_query (str): User's legal query
        vectorstore: FAISS vector store
        llm: LLM instance
        case_summary (str): Running summarized memory of the case

    Returns:
        response (str): Assistant reply
        updated_case_summary (str): Updated memory summary
    """

    # 1. Detect law domain
    detected_domain = detect_law_domain(user_query)

    # 2. Create retriever (domain-aware if possible)
    if detected_domain:
        retriever = vectorstore.as_retriever(
            search_kwargs={
                "k": 6,
                "filter": {"law_type": detected_domain}
            }
        )
    else:
        retriever = vectorstore.as_retriever(
            search_kwargs={"k": 6}
        )

    # 3. Retrieve relevant legal documents
    docs = retriever.invoke(user_query)


    # 4. Build legal context
    context = "\n\n".join(
        f"[{d.metadata.get('law_type')} | {d.metadata.get('Section_Article')}]\n{d.page_content}"
        for d in docs
    )

    # 5. Prompt construction
    prompt = f"""
You are a professional Indian legal assistant.

Case summary so far:
{case_summary}

User query:
{user_query}

Relevant legal provisions:
{context}

Instructions:
- Answer strictly based on Indian law
- Mention relevant sections
- Do not hallucinate
- If information is insufficient, ask clarifying questions
"""

    # 6. LLM call
    response = llm.invoke(prompt).content

    # 7. Update case summary (memory)
    updated_case_summary = update_case_summary(
        llm,
        case_summary,
        user_query,
        response
    )

    return response, updated_case_summary


In [20]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.load_local(
    folder_path="law_faiss_store",
    embeddings=embeddings,
    allow_dangerous_deserialization=True
)


In [23]:
case_summary = ""


In [24]:
response, case_summary = legal_assistant_chat(
    user_query="My employer terminated me without notice",
    vectorstore=vectorstore,
    llm=llm,
    case_summary=case_summary
)

print(response)


Based on the legal provisions provided:

1.  **For Government Employees (Civil Servants):**
    If you are a member of a civil service of the Union or a State, or hold a civil post under the Union or a State, **Article 311 of the Indian Constitution** provides significant protection.
    *   **Article 311(1)** states that you cannot be dismissed or removed by an authority subordinate to that by which you were appointed.
    *   **Article 311(2)** mandates that you cannot be dismissed, removed, or reduced in rank *except after an inquiry* where you have been informed of the charges against you and given a reasonable opportunity of being heard in respect of those charges. Termination without such a due process would be unconstitutional for a civil servant.

2.  **For Private Sector Employees or Other Cases:**
    The provided provisions **do not directly address** the issue of "termination without notice" for employees in the private sector or general employment contracts.
    *   **[CON

In [25]:
response, case_summary = legal_assistant_chat(
    user_query="It was a private company and I worked for 3 years",
    vectorstore=vectorstore,
    llm=llm,
    case_summary=case_summary
)

print(response)


Thank you for clarifying that you were employed by a private company for 3 years.

Given this information:
1.  **Article 311 of the Indian Constitution**, which offers significant protection against dismissal for government employees (civil servants), **does not apply** to your situation as you were a private sector employee.

2.  The legal provisions provided in the prompt ([CONTRACT_LAWS | Illustrations] and [INDIAN_CRIMINAL_LAW_2]) are **not relevant** to your case of termination without notice from a private company.
    *   The sections from `CONTRACT_LAWS` (e.g., 257-266, SCHDUL) are either repealed or pertain to partnership law, not general employment contracts.
    *   The provisions from `INDIAN_CRIMINAL_LAW_2` relate to criminal procedure, service of summons, and forms for attachment of property, which are entirely distinct from employment termination matters.

**Legal Assessment for Private Sector Termination:**
In the private sector in India, the terms and conditions of emp